In [1155]:
full_info = {}

In [1156]:
import nltk
from nltk.corpus import stopwords
stop = stopwords.words('english')

In [1157]:
import docx

def getText(filename):
    doc = docx.Document(filename)
    fullText = []
    for para in doc.paragraphs:
        txt = para.text.encode('ascii', 'ignore')
        fullText.append(txt)
    return '\n'.join(fullText)

In [1158]:
filename = '../docs/doc_1.docx'

In [1159]:
document = getText(filename)

In [1160]:
import re
lines_ini = re.split('    |\n',document)
lines_ini = [el.strip(',') for el in lines_ini]

In [1161]:
lines = [nltk.word_tokenize(el) for el in lines_ini]
lines = [nltk.pos_tag(el) for el in lines]
lines = [x for x in lines if x != []]
lines = lines[1:-3]

In [1162]:
lines_ini

['',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '   Date: dd/mm/yyyy ',
 ' ',
 'To',
 'Mr. Md Azhar',
 ' ',
 ' \t \t \t \tSub: Offer Letter  ',
 ' ',
 'Dear Ms ',
 'Subject to the terms contained in this letter of appointment, we are pleased to appoint you as  Software Engineer Trainee effective from 25/09/2018. ',
 '1.Commencement of  employment ',
 'Your employment with the Company shall commence on 20/09/2018, subject you being medically fit and validation / verification of your educational qualification. ',
 ' ',
 '2.Placement  compensation \t ',
 'Compensation will be governed by the rules of the Company on the subject, as applicable and/or amended hereafter.The Company expects you to keep the salary details confidential at all times. This compensation will be reviewed from time to time depending upon the successful completion of tasks assigned to you.The Company reserves the right to withdraw 

In [1163]:
grammer = r'NAME: {^<NN.*><NN.*>?<NN.*>?$}'
chunkParser = nltk.RegexpParser(grammer)

In [1164]:
nameHits = []
for tagged_tokens in lines:
    chunked_tokens = chunkParser.parse(tagged_tokens)
    for subtree in chunked_tokens.subtrees():
        if subtree.label() == 'NAME':
            nameHits.append(' '.join([i[0] for i in subtree.leaves()]))

In [1165]:
full_info['Candidate Name'] = nameHits[0]
if full_info['Candidate Name'].find('Date')!=-1:
    full_info['Candidate Name'] = nameHits[1]
full_info['offerer Name'] = nameHits[-1]
if nameHits[-1] == 'Hiring Manager':
    full_info['offerer Name'] = nameHits[-2]

In [1166]:
for l in lines_ini:
    if l.lower().find('dear')!= -1 and len(l) > 8:
        full_info['Candidate Name'] = l[(l.lower().find('dear') + 5):]
        break

In [1167]:
import spacy
nlp = spacy.load('en')

def extract_join_dt_sal_org(lines):
    dates = []
    money = []
    org = []
    for line in lines:
        doc=nlp(unicode(line))
        for ent in doc.ents:
            if ent.label_ == 'ORG':
                    org.append(str(ent.text))
            if ent.label_ == 'MONEY':
                    money.append(str(ent.text))
            if ent.label_ == 'DATE':
                    match = re.search(r'(^\d{1,2}\D.*\d{4}$)',ent.text.strip())
                    if match != None:
                        dates.append(str(ent.text))
    
    if not dates:
        dates = ['not found']
    if not money:
        money = ['not found']
    if not org:
        org = ['not found']
    return dates[-1],money[0],org[0]

In [1168]:
join_date,salary,org=extract_join_dt_sal_org(lines_ini)
full_info['Joining date'] = join_date
full_info['Salary'] = salary
full_info['Organisation Name'] = org

In [1169]:
import find_job_titles
from find_job_titles import FinderAcora
b=FinderAcora()
gen=FinderAcora.find_raw(b,unicode(document))
designation=[]
for l in gen:
    designation.append(l)
    break

In [1170]:
if not designation:
    full_info['Designation'] = 'Not found'
else:
    des= designation[0]
    d = document[des.start-15:des.end+15]
    if document.find('Trainee')!=-1:
        ind = document.find('Trainee')
        d = document[ind-20:ind+27]
    if d == '':
        d = document[des.start:des.end+15]
        
    d = nltk.word_tokenize(str(d))
    d = nltk.pos_tag(d)
    grammer = r'DESIGNATION: {<NNP>*}'
    chunkParser = nltk.RegexpParser(grammer)
    desHits = []
    chunked_tokens = chunkParser.parse(d)
    for subtree in chunked_tokens.subtrees():
        if subtree.label() == 'DESIGNATION':
            word=' '.join([i[0] for i in subtree.leaves()])
            if document.find('Trainee')!=-1:
                desHits.append(word)
            elif des.match in word:
                desHits.append(word)
    if not desHits:
        full_info['Designation'] = 'Not found'
    else:
        full_info['Designation'] = desHits[0]

In [1171]:
if ((full_info['Designation']).strip() == (full_info['Organisation Name']).strip()):
    full_info['Organisation Name'] = 'Not Found'

In [1172]:
full_info

{'Candidate Name': 'Mr. Md Azhar',
 'Designation': 'Software Engineer Trainee',
 'Joining date': '20/09/2018',
 'Organisation Name': 'Not Found',
 'Salary': 'not found',
 'offerer Name': 'David watson'}